In [30]:
import pandas as pd
data = pd.read_csv('../data/processed/data_preparada.csv')
data.head()

,google_index,domain_registration_length,page_rank,safe_anchor,nb_www,ratio_extRedirection,avg_words_raw,ratio_digits_url,nb_dots,ratio_digits_host,...,avg_word_path,length_words_raw,phish_hints,shortest_word_path,nb_hyphens,web_traffic,char_repeat,links_in_tags,nb_slash,status
0,1,45,4,0.0,1,0.875000,5.750000,0.000000,3,0.0,...,4.500000,4,0,3,0,0,4,80.000000,3,0
1,1,77,2,100.0,0,0.000000,15.750000,0.220779,1,0.0,...,14.666667,4,0,2,0,0,4,100.000000,5,1
2,1,14,0,100.0,0,0.000000,8.250000,0.150794,4,0.0,...,8.142857,12,0,2,1,5828815,2,100.000000,5,1
3,0,62,3,62.5,0,0.250000,5.000000,0.000000,2,0.0,...,0.000000,1,0,0,0,107721,0,100.000000,2,0
4,0,224,6,0.0,1,0.537037,6.333333,0.000000,2,0.0,...,7.000000,6,0,4,2,8725,3,76.470588,5,0


In [28]:
import re

def analisis_url(url):
    fila_url = []

    # Fila 1 (google_index): Indica si la página web aparece en el índice de Google.
    if re.search(r"\bgoogle\b", url):
        fila_url.append(1)
    else:
        fila_url.append(0)

    # Fila 2 (domain_registration_length): Longitud del registro del dominio.
    longitud_dominio = len(re.findall(r"\.[a-z]{2,}$", url))
    fila_url.append(longitud_dominio)

    # Fila 3 (page_rank): Ranking de la página web.
    rank = re.findall(r"\brank=(\d+)\b", url)
    if rank:
        fila_url.append(int(rank[0]))
    else:
        fila_url.append(0)

    # Fila 4 (safe_anchor): Indica si los enlaces de la página web utilizan atributos "rel" para evitar la apertura de nuevas pestañas.
    if re.search(r'<a\s[^>]*\brel="noopener noreferrer"\b', url):
        fila_url.append(1)
    else:
        fila_url.append(0)

    # Fila 5 (nb_www): Indica si la URL contiene "www" en el nombre de host.
    if re.search(r"\bwww\b", url):
        fila_url.append(1)
    else:
        fila_url.append(0)

    # Fila 6 (ratio_extRedirection): Proporción de redirecciones externas en la página web.
    num_links = re.findall(r"<a\s[^>]*href=", url)
    num_redirecciones_externas = len(re.findall(r"<meta\s[^>]*http-equiv=\"refresh\"[^>]*url=(?:(?!"+url+").)*$", url, re.MULTILINE))
    if len(num_links) > 0:
        fila_url.append(num_redirecciones_externas / len(num_links))
    else:
        fila_url.append(0)

    # Fila 7 (avg_words_raw): Longitud promedio de las palabras en la ruta de la URL sin caracteres especiales.
    ruta = re.findall(r"(?:https?://[\w\.-]+/)([\w\.-]+)/?", url)
    if ruta:
        palabras = re.findall(r"\b\w+\b", ruta[0])
        if len(palabras) > 0:
            fila_url.append(sum(len(palabra) for palabra in palabras) / len(palabras))
        else:
            fila_url.append(0)
    else:
        fila_url.append(0)

    # Fila 8 (ratio_digits_url): Proporción de dígitos en la URL.
    num_digitos = len(re.findall(r"\d", url))
    if len(url) > 0:
        fila_url.append(num_digitos / len(url))
    else:
        fila_url.append(0)

    # Fila 9 (nb_dots): Número de puntos en la URL.
    fila_url.append(url.count("."))

    # Fila 10 (ratio_digits_host): Proporción de dígitos en el nombre de host de la URL.
    host = re.findall(r"https?://([^/]+)", url)
    if host:
        num_digitos = len(re.findall(r"\d", host[0]))
        if len(host[0]) > 0:
            fila_url.append(num_digitos / len(host[0]))
        else:
            fila_url.append(0)
    else:
        fila_url.append(0)

    # Fila 11 (domain_age): Antigüedad del dominio.
    antiguedad = re.findall(r"\bage=(\d+)\b", url)
    if antiguedad:
        fila_url.append(int(antiguedad[0]))
    else:
        fila_url.append(0)

    # Fila 12 (domain_in_title): Indica si el nombre de dominio está incluido en el título de la página web.
    if re.search(r"<title>(?:.*?)\b" + re.escape(host[0]) + r"\b(?:.*?)</title>", url, re.IGNORECASE):
        fila_url.append(1)
    else:
       fila_url.append(0)

    # Fila 13 (ratio_extHyperlinks): Proporción de hipervínculos externos en la página web.
    num_links = len(re.findall(r"<a\s[^>]*href=", url))
    num_links_externos = len(re.findall(r"<a\s[^>]*href=\"(?:https?:)?//(?:(?!" + re.escape(host[0]) + r").)*\"", url))
    if num_links > 0:
        fila_url.append(num_links_externos / num_links)
    else:
        fila_url.append(0)

    # Fila 14 (longest_words_path): Longitud de la palabra más larga en la URL sin caracteres especiales.
    palabras = re.findall(r"\b\w+\b", url)
    if palabras:
        palabra_mas_larga = max(palabras, key=len)
        fila_url.append(len(palabra_mas_larga))
    else:
        fila_url.append(0)

    # Fila 15 (shortest_word_host): Longitud de la palabra más corta en el nombre de host de la URL sin caracteres especiales.
    host = re.findall(r"https?://([^/]+)", url)
    if host:
        palabras = re.findall(r"\b\w+\b", host[0])
        if palabras:
            palabra_mas_corta = min(palabras, key=len)
            fila_url.append(len(palabra_mas_corta))
        else:
            fila_url.append(0)
    else:
        fila_url.append(0)

    # Fila 16 (nb_hyperlinks): Número de hipervínculos en la página web.
    fila_url.append(num_links)

    # Fila 17 (ratio_intHyperlinks): Proporción de hipervínculos internos en la página web.
    num_links_internos = len(re.findall(r"<a\s[^>]*href=\"(?:/|\.\./|javascript:|#).*\"", url))
    if num_links > 0:
        fila_url.append(num_links_internos / num_links)
    else:
        fila_url.append(0)

    # Fila 18 (longest_words_path): Longitud de la palabra más larga en la ruta de la URL sin caracteres especiales.
    ruta = re.findall(r"(?:https?://[\w\.-]+/)([\w\.-]+)/?", url)
    if ruta:
        palabras = re.findall(r"\b\w+\b", ruta[0])
        if palabras:
            palabra_mas_larga = max(palabras, key=len)
            fila_url.append(len(palabra_mas_larga))
        else:
            fila_url.append(0)
    else:
        fila_url.append(0)

    # Fila 19 (length_hostname): Longitud del nombre de host en la URL.
    host = re.findall(r"https?://([^/]+)", url)
    if host:
        fila_url.append(len(host[0]))
    else:
        fila_url.append(0)

    # Fila 20 (avg_word_path): Longitud promedio de las palabras en la ruta de la URL sin caracteres especiales.
    ruta = re.findall(r"(?:https?://[\w\.-]+/)([\w\.-]+)/?", url)
    if ruta:
        palabras = re.findall(r"\b\w+\b", ruta[0])
        if len(palabras) > 0:
            fila_url.append(sum(len(palabra) for palabra in palabras) / len(palabras))
        else:
            fila_url.append(0)
    else:
        fila_url.append(0)

    # Fila 21 (length_words_raw): Longitud de la URL sin caracteres especiales.
    raw_url = re.findall(r"https?://([^/]+)(/.*)?", url)
    if raw_url:
        url_sin_caracteres_especiales = re.sub(r"\W+", "", raw_url[0][0]) + re.sub(r"\W+", "", raw_url[0][1]) if raw_url[0][1] else re.sub(r"\W+", "", raw_url[0][0])
        fila_url.append(len(url_sin_caracteres_especiales))
    else:
        fila_url.append(0)

    # Fila 22 (phish_hints): Indica si la URL contiene palabras clave asociadas con phishing.
    phishing_keywords = ["login", "banking", "password", "paypal", "amazon", "security", "verify", "account"]
    if any(keyword in url.lower() for keyword in phishing_keywords):
        fila_url.append(1)
    else:
        fila_url.append(0)

    # Fila 23 (shortest_word_path): Longitud de la palabra más corta en la ruta de la URL sin caracteres especiales.
    ruta = re.findall(r"(?:https?://[\w\.-]+/)([\w\.-]+)/?", url)
    if ruta:
        palabras = re.findall(r"\b\w+\b", ruta[0])
        if palabras:
            palabra_mas_corta = min(palabras, key=len)
            fila_url.append(len(palabra_mas_corta))
        else:
            fila_url.append(0)
    else:
        fila_url.append(0)

    # Fila 24 (nb_hyphens): Número de guiones en la URL.
    fila_url.append(url.count("-"))

    # Fila 25 (web_traffic): Tráfico de la página web.
    trafico = re.findall(r"\btraffic=(\d+)\b", url)
    if trafico:
        fila_url.append(int(trafico[0]))
    else:
        fila_url.append(0)

    # Fila 26 (char_repeat): Indica si hay caracteres repetidos en la URL.
    if re.search(r"(.)\1{2,}", url):
        fila_url.append(1)
    else:
        fila_url.append(0)

    # Fila 27 (links_in_tags): Indica si hay enlaces en las etiquetas HTML de la página web.
    if re.search(r"<a\s", url):
        fila_url.append(1)
    else:
        fila_url.append(0)

    # Fila 28 (nb_slash): Número de barras diagonales en la URL.
    fila_url.append(url.count("/"))

    return fila_url

In [29]:
analisis_url('http://www.crestonwood.com/router.php')

[0,
 1,
 0,
 0,
 1,
 0,
 4.5,
 0.0,
 3,
 0.0,
 0,
 0,
 0,
 11,
 3,
 0,
 0,
 6,
 19,
 4.5,
 26,
 0,
 3,
 0,
 0,
 1,
 0,
 3]